<a href="https://colab.research.google.com/github/ihasdapie/Colab-Notebooks/blob/master/butternut_colab_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%capture
!pip install transformers
!pip install flask-ngrok
!pip install flask==0.12.2 # for compatability with flask-ngrok

import numpy as np
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import flask
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

In [2]:
!rm -rf butternut_backend
!git clone https://github.com/btrnt/butternut_backend.git

import butternut_backend.src.gltr as gltr


Cloning into 'butternut_backend'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 39 (delta 15), reused 29 (delta 9), pack-reused 0
Unpacking objects: 100% (39/39), done.


In [3]:
gmdl = gltr.LM()

Loaded GPT-2 model!


In [4]:
gmdl.check_probabilities("a string")

{'bpe_strings': ['<|endoftext|>', 'a', 'Ġstring'],
 'pred_topk': [[('Ċ ', 0.0623)], [('.', 0.05815)]],
 'real_topk': [(65, 0.00192), (173, 0.0007)]}

In [5]:
app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

@app.route('/gp', methods=['GET', 'POST']) # (g)et_(p)robability
def gp():
    global test_probs
    # just return a dummy run for now
    # raw_text = "How much wood would a woodchuck chuck if a woodchuck could chuck wood?"

    if request.method == 'POST':  #this block is only entered when the form is submitted
        raw_text = request.form.get('text')
        payload = gmdl.check_probabilities(raw_text, topk=1)
        del payload['pred_topk']      
        return jsonify(payload)
        
    s = """<form method="POST">
                  Text: <input type="text" name="text"><br>
                  <input type="submit" value="Submit"><br>
              </form>"""
    return s

In [8]:
run_with_ngrok(app)
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d212a65cdd62.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Jan/2021 01:14:15] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [10/Jan/2021 01:14:15] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jan/2021 01:14:17] "GET /gp HTTP/1.1" 200 -
